In [1]:
# importing required libraries
import nltk
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix,classification_report

In [2]:
# read the dataset
train = pd.read_csv("./train.txt", delimiter=';', header=None, names=['sentence','label'])
test = pd.read_csv("./test.txt", delimiter=';', header=None, names=['sentence','label'])
val = pd.read_csv("./val.txt", delimiter=';', header=None, names=['sentence','label'])

In [14]:
df_data = pd.concat([train, test,val])
df_data

pandas.core.frame.DataFrame

In [4]:
df_data.to_csv (r'exportdata.txt', index=False)
dt_data =  pd.read_csv("exportdata.txt")
dt_data.head

<bound method NDFrame.head of                                                 sentence    label
0                                i didnt feel humiliated  sadness
1      i can go from feeling so hopeless to so damned...  sadness
2       im grabbing a minute to post i feel greedy wrong    anger
3      i am ever feeling nostalgic about the fireplac...     love
4                                   i am feeling grouchy    anger
...                                                  ...      ...
19995  im having ssa examination tomorrow in the morn...  sadness
19996  i constantly worry about their fight against n...      joy
19997  i feel its important to share this info for th...      joy
19998  i truly feel that if you are passionate enough...      joy
19999  i feel like i just wanna buy any cute make up ...      joy

[20000 rows x 2 columns]>

In [5]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english', ngram_range=(1,1), tokenizer = token.tokenize)
text = cv.fit_transform(dt_data['sentence'])

In [6]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
sw = set(stopwords.words('english'))
ps = PorterStemmer()
cv = CountVectorizer(stop_words='english', ngram_range=(1,1), tokenizer = token.tokenize)
text = cv.fit_transform(dt_data['sentence'])

In [7]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(text,dt_data['label'], test_size=0.30, random_state=5)

# get the shape of train and test split.
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((14000, 16798), (6000, 16798), (14000,), (6000,))

In [8]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)

MultinomialNB()

In [9]:
predicted = mnb.predict(X_test)

In [10]:
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix,classification_report

In [11]:
acc_score = metrics.accuracy_score(predicted,y_test)
prec_score = precision_score(y_test,predicted, average='macro')
recall = recall_score(y_test, predicted,average='macro')
f1 = f1_score(y_test,predicted,average='macro')
matrix = confusion_matrix(y_test,predicted)

In [12]:
print(str('Accuracy: '+'{:04.2f}'.format(acc_score*100))+'%')
print(str('Precision: '+'{:04.2f}'.format(prec_score*100))+'%')
print(str('Recall: '+'{:04.2f}'.format(recall*100))+'%')
print('F1 Score: ',f1)
print(matrix)

Accuracy: 77.53%
Precision: 82.71%
Recall: 58.59%
F1 Score:  0.6191209661316305
[[ 512   14   92    2  179    0]
 [  26  430   98    2  144    1]
 [  12   12 1890   23   95    0]
 [   7    5  241  162   74    0]
 [  13   11   83    2 1644    1]
 [   4   29   92    2   84   14]]


In [17]:
@app.route('/predict', methods=['POST'])
def predict():
    result = mnb.predict(cv.transform(request.json['data']))
    return result